# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [9]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

total = 0
for sqrt_value in generator:
    print(sqrt_value)
    total += sqrt_value
print(f'{total=}')

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
total=32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [1]:
import dlt
import duckdb


In [26]:

pipeline = dlt.pipeline(destination="duckdb", dataset_name="people")
info = pipeline.run(
    people_1(),
    table_name="people",
    write_disposition="replace",
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\kuroc\Google Drive\Projects\data-engineering-zoomcamp2024-homework\dlt_workshop\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707774426.4115057 is LOADED and contains no failed jobs


In [27]:

info = pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append",
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.52 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\kuroc\Google Drive\Projects\data-engineering-zoomcamp2024-homework\dlt_workshop\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707774428.1458979 is LOADED and contains no failed jobs


In [28]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
print('Loaded tables: ')
display(conn.sql("use people; show tables;"))


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [29]:
# print("\n\n\n Sum of all ages of people.")
ages = conn.sql("SELECT sum(age) FROM people.people").df()
display(ages)
# delete(conn)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [6]:
# #Install the dependencies
# %%capture
# !pip install dlt[duckdb]

In [23]:
# rerun cell with  people_1() replace

info = pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="merge",
    primary_key="ID"
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///c:\Users\kuroc\Google Drive\Projects\data-engineering-zoomcamp2024-homework\dlt_workshop\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707774326.078502 is LOADED and contains no failed jobs


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [24]:
ages = conn.sql("SELECT sum(age) FROM people.people").df()
display(ages)

,sum(age)
0,266.0
